# Submit

Tools that run on the hubs can use only modest computing resources because they run in a container on a shared host.  Many simulations require large amounts of memory and/or CPU time.  These tools require access to external HPC or grid resources.  

**submit** takes a user command and executes it remotely. The objective is to allow the user to issue a command in the same manner as a locally executed command. Multiple submission mechanisms are available for run dissemination. A set of steps are executed for each run submission:

* Destination site is selected
* A wrapper script is generated for remote execution.

* If needed a batch system description file is generated.
* Input files for a run are gathered and transferred to the remote site. Transferred files include the wrapper and batch description scripts.
* The wrapper script is executed remotely.
* Progress of the remote run is monitored until completion.
* Output files from the run are returned to the dissemination point.


## Contents

1. [**Introduction to Submit**](#Introduction-to-Submit) 
    * [Learning by Doing](#Learning-by-Doing)
    * [Submit with Parameters](#Submit-with-Parameters)
    * [Using runCommand](#Using-runCommand)
    * [Submit status Email](#Submit-status-email)
    * [Using SubmitCommand class](#Using-SubmitCommand-class)
    

2. **Submit and Jupyter**
    * [Submit UI widget](../GUI/hublib/SubmitWidget.ipynb) Run the hub submit command in a widget. Uses local caching of results.
    * [Submit-Parametric](../GUI/hublib/SubmitWidget-Parametric.ipynb) A parametric example that runs many jobs.
    * [Submit-NoCache](../GUI/hublib/SubmitWidget-NoCache.ipynb) Submit widget with caching disabled.

## Introduction to Submit

The submit command allows hub users, or hub tools, to run commands on remote computing resources. The syntax for this command is as follows:

```bash
> submit --help
Usage: submit [options]

Options:
  -h, --help            Report command usage. Optionally request listing of
                        managers, tools, venues, or examples.
  -l, --local           Execute command locally
  --status              Report status for runs executing remotely.
  -k, --kill            Kill runs executing remotely.
  --venueStatus         Report venue status.
  -v, --venue           Remote job destination
  -i, --inputfile       Input file
  -p, --parameters      Parameter sweep variables. See examples.
  -d, --data            Parametric variable data - csv format
  -s SEPARATOR, --separator=SEPARATOR
                        Parameter sweep variable list separator
  -n NCPUS, --nCpus=NCPUS
                        Number of processors for MPI execution
  -N PPN, --ppn=PPN     Number of processors/node for MPI execution
  -w WALLTIME, --wallTime=WALLTIME
                        Estimated walltime hh:mm:ss or minutes
  -e, --env             Variable=value
  --runName=RUNNAME     Name used for directories and files created during the
                        run. Restricted to alphanumeric characters
  -m, --manager         Multiprocessor job manager
  -r NREDUNDANT, --redundancy=NREDUNDANT
                        Number of indentical simulations to execute in
                        parallel
  -M, --metrics         Report resource usage on exit
  --detach              Detach client after launching run
  --attach=ATTACHID     Attach to previously detached started server
  -W, --wait            Wait for reduced job load before submission
  -Q, --quota           Enforce local user quota on remote execution host
  -q, --noquota         Do not enforce local user quota on remote execution
                        host
  --tailStdout          Periodically report tail of stdout file.
  --tailStderr          Periodically report tail of stderr file.
  --tail                Periodically report tail of application file.
  --progress            Show progress method. Choices are auto, curses,
                        submit, text, pegasus, or silent.
```


# Learning by Doing

This notebook will demonstrate how to use submit.  First, it will show command line usage that will not require Python.  Then, it will show some examples that use a Python convenience function to run submit.


## Venues

Submit can submit to different hosts or clusters. You can get a list for your hub by asking submit:


In [1]:
!submit --help venues


Currently available VENUES are:
   ccr-ghub
   ccr-ghub-8cores
   ccr-ghub-debug
   ccr-ghub-script
   submithost
   u2-grid
   u2-grid-debug


<div class="alert alert-warning">
How do we know what the default venue is?
How should we choose the venue to use?
</div>

You may guess that the GHub gateway uses venues with `ghub` in the name. For that gateway, `ccr-ghub` is the best choice for most purposes. When in doubt, ask!

### What commands can I submit?

#### Submitting locally with --local

Any executable can be used when submitting locally.  Local submit runs in the current
execution host in the current container, so there are few reasons to use it.  However, it
does run quickly, so it is useful for quick tests such as the ones we run in this notebook.

Here's a very quick example:

In [2]:
!submit --local echo hi

hi


#### Submitting to outside resources

Only executables staged in the Hub's /apps directory can be submitted to the grid (outside computing resources). We'll see an example of that in the next section. 

## Submit with Parameters

The submit command accepts various different parameters and input files. Some examples of syntax for submit parameters follow:

`submit -p @@cap=10pf,100pf,1uf sim.exe @:indeck`

    Submit 3 jobs. The @:indeck means "use the file indeck as a template
    file." Substitute the values 10pf, 100pf, and 1uf in place of @@cap within the
    file. Send off one job for each of the values and bring back the results.

`submit -p @@vth=0:0.2:5 -p @@cap=10pf,100pf,1uf sim.exe @:indeck`

    Submit 78 jobs. The parameter @@vth goes from 0 to 5 in steps of 0.2,
    so there are 26 values for @@vth. For each of those values, the parameter
    @@cap changes from 10pf to 100pf to 1uf. 26 x 3 = 78 jobs total. Again
    @:indeck is treated as a template, and the values are substituted in place of
    @@vth and @@cap in that file.

`submit -p params sim.exe @:indeck`

    In this case, parameter definitions are taken from the file named
    params instead of the command line. The file might have the following
    contents:

        # paramters for my job submission
        parameter @@vth=0:0.2:5
        parameter @@cap = 10pf,100pf,1uf

`submit -p "params;@@num=1-10;@@color=blue" job.sh @:job.data`

    Here, the semicolon separates
    the parameters value into three parts. The first says to load parameters from
    a file params. The next part says add an additional parameter @@num that goes
    from 1 to 10. The last part says add an additional parameter @@color with a
    single value blue. The parameters @@num and @@color cannot override anything
    defined within params; they must be new parameter names.

`submit -d input.csv sim.exe @:indeck`

    Takes parameters from the data file input.csv, which must be in comma-
    separated value format. The first line of this file may contain a series of
    @@param names for each of the columns. Whitespace is significant for all
    values entered in the csv file. If it doesn't, then the columns are assumed to
    be called @@1, @@2, @@3, etc. Each of the remaining lines represents a set of
    parameter values for one job; if there are 100 such lines, there will be 100
    jobs. For example, the file input.csv might look like this:

        @@vth,@@cap
        1.1,1pf
        2.2,1pf
        1.1,10pf
        2.2,10pf

    Parameters are substituted as before into template files such as
    @:indeck.

`submit -d input.csv -p "@@doping=1e15-1e17 in 30 log" sim.exe @:infile`

    Takes parameters from the data file input.csv, but also adds another
    parameter @@doping which goes from 1e15 to 1e17 in 30 points on a log scale.
    For each of these points, all values in the data file will be executed. If the
    data file specifies 50 jobs, then this command would run 30 x 50 = 1500 jobs.

`submit -d input.csv -i @:extra/data.txt sim.exe @:indeck`

    In addition to the template indeck file, send along another file
    extra/data.txt with each job, and treat it as a template too.

`submit -s / -p @@address=23 Main St.,Hometown,Indiana/42 Broadway,Hometown,Indiana -s , -p @@color=red,green,blue job.sh @:job.data`

    Change the separator to slash when defining the addresses, then change
    back to comma for the @@color parameter and any remaining arguments. We
    shouldn't have to change the separator often, but it might come in handy if
    the value strings themselves have commas.

`submit -p @@num=1:1000 sim.exe input@@num`

    Submit jobs 1,2,3,...,1000. Parameter names such as @@num are
    recognized not only in template files, but also for arguments on the command
    line. In this case, the numbers 1,2,3,...,1000 are substituted into the file
    name, so the various jobs take their input from "input1", "input2", ...,
    "input1000".

`submit -p @@file=glob:indeck* sim.exe @@file`

    Look for files matching indeck* and use the list of names as the
    parameter @@file. Those values could be substituted into other template files,
    or used on the command line as in this example. Suppose the directory contains
    files indeck1, indeck10, and indeck2.  The glob option will order the files in
    a natural order: indeck1, indeck2, indeck10.  This example would launch three
    jobs using each of those files as input for the job.

`submit -p @@file=globnat:indeck* sim.exe @@file`

    This option has been deprecated.  The functionality is now available with the glob option.

## Submit with parameters: examples

Let's try the echo program with a list of different input parameters.  To see progress, we need to use "text" or "submit".  Other options won't work well with Jupyter.


In [3]:
# We need this because this example is running in a read-only directory. We use the /tmp directory in the running container:
import os
try:
    os.mkdir('/tmp/submit')
except:
    pass
os.chdir('/tmp/submit')

Now, for the `echo` example:

In [4]:
!submit --local --progress text -p @@name=hub1,hub2,hub3 echo @@name

Simulations complete. Results are stored in directory /tmp/submit/00032650


Notice how the outputs were written to subdirectories in a directory that was created for us.
If we want to access those results more conveniently, it would be best to tell submit what name to use for the directory.
We can do this by using the **--runName** parameter.
<div class="alert alert-info">
If the directory exists, submit will complain and exit immediately.

</div>

In [5]:
!rm -rf echotest
!submit --local --runName=echotest --progress submit -s, -p @@name=hub1,hub2,hub3 echo @@name

=SUBMIT-PROGRESS=> aborted=0 finished=0 failed=0 executing=0 waiting=0 setting_up=0 setup=3 %done=0.00 timestamp=1625662207.6
=SUBMIT-PROGRESS=> aborted=0 finished=0 failed=0 executing=1 waiting=0 setting_up=0 setup=2 %done=0.00 timestamp=1625662215.9
=SUBMIT-PROGRESS=> aborted=0 finished=1 failed=0 executing=0 waiting=0 setting_up=0 setup=2 %done=33.33 timestamp=1625662216.0
=SUBMIT-PROGRESS=> aborted=0 finished=1 failed=0 executing=1 waiting=0 setting_up=0 setup=1 %done=33.33 timestamp=1625662216.3
=SUBMIT-PROGRESS=> aborted=0 finished=2 failed=0 executing=0 waiting=0 setting_up=0 setup=1 %done=66.67 timestamp=1625662216.4
=SUBMIT-PROGRESS=> aborted=0 finished=2 failed=0 executing=1 waiting=0 setting_up=0 setup=0 %done=66.67 timestamp=1625662216.6
=SUBMIT-PROGRESS=> aborted=0 finished=3 failed=0 executing=0 waiting=0 setting_up=0 setup=0 %done=100.00 timestamp=1625662216.7


Once submit finishes, we can see the standard output for each job in the "echotext" subdirectory.  Standard output  will be in echotest_{jobnum}.stdout.

In [6]:
!ls -lR echotest

echotest:
total 4
drwxr-xr-x 2 jsperhac public  32 Jul  7 12:50 01
drwxr-xr-x 2 jsperhac public  32 Jul  7 12:50 02
drwxr-xr-x 2 jsperhac public  32 Jul  7 12:50 03
drwxr-xr-x 3 jsperhac public  21 Jul  7 12:50 jsperhac
-rw-r--r-- 1 jsperhac public 197 Jul  7 12:50 parameterCombinations.csv

echotest/01:
total 4
-rw-rw-r-- 1 jsperhac public 5 Jul  7 12:50 echotest_01.stdout

echotest/02:
total 4
-rw-rw-r-- 1 jsperhac public 5 Jul  7 12:50 echotest_02.stdout

echotest/03:
total 4
-rw-rw-r-- 1 jsperhac public 5 Jul  7 12:50 echotest_03.stdout

echotest/jsperhac:
total 0
drwxr-xr-x 3 jsperhac public 22 Jul  7 12:50 pegasus

echotest/jsperhac/pegasus:
total 0
drwxr-xr-x 2 jsperhac public 6 Jul  7 12:50 00032651

echotest/jsperhac/pegasus/00032651:
total 0


So let's look at those output files:

In [7]:
for i in range(1,4):
    with open("echotest/%02d/echotest_%02d.stdout" % (i, i), 'r') as f:
        print("Run %02d: %s" % (i, f.read()))

Run 01: hub1

Run 02: hub2

Run 03: hub3



In [8]:
!cat echotest/parameterCombinations.csv

# Using runCommand

Using the exclamation point(!) to run shell commands from Jupyter is fine for many cases.  But often you want to call submit from a Python function or check the return value.  Hublib provides some very flexible Python functions to do this  https://hubzero.github.io/hublib/cmd.html

### Managers

In order to run an example Python3 code on our remote resource, we will use a submit manager (here, called `ccr-vhub-python`) which loads a Python3 Anaconda installation, with the needed packages, on our remote resource. This is not typically necessary, but you'll see it in the example below. 

Here's the command for identifying managers that are available for use with submit:

In [21]:
!submit --help managers


Currently available MANAGERs are:
   ccni-bgl-CO
   ccni-bgl-VN
   ccni-opteron_lammps
   ccr-vhub-python
   mpi
   mpich
   mpirun
   parallel
   pegasus
   puffin-intel-12.1
   python-2.7.2
   sbbnl-bgl-CO
   sbbnl-bgl-VN
   sbbnl-bgp-DUAL
   sbbnl-bgp-SMP
   sbbnl-bgp-VN
   serial
   titan-manager


### The runCommand() call
Note that we specify `-v` with the remote computing resource, or [venue](#Venues), name, and `-m` with the [manager](#Managers) name. Other [parameters](#Submit-with-Parameters) are described above. 

Refer to the documentation for all the details of the syntax.

In [31]:
from hublib.cmd import runCommand
!rm -rf runtest
res, stdout, stderr = runCommand("submit -v ccr-ghub \
                                 -m ccr-vhub-python \
                                 --runName=runtest \
                                 --progress submit \
                                 -p @@Vin=1,2 \
                                 /apps/pegtut/current/examples/capacitor_voltage/sim1.py  \
                                 --Vin @@Vin")

=SUBMIT-PROGRESS=> aborted=0 finished=0 failed=0 executing=0 waiting=2 setting_up=0 setup=0 %done=0.00 timestamp=1625682283.8
=SUBMIT-PROGRESS=> aborted=0 finished=0 failed=0 executing=2 waiting=0 setting_up=0 setup=0 %done=0.00 timestamp=1625682319.0
=SUBMIT-PROGRESS=> aborted=0 finished=2 failed=0 executing=0 waiting=0 setting_up=0 setup=0 %done=100.00 timestamp=1625682354.2
Simulations complete. Results are stored in directory /home/vhub/jsperhac/data/sessions/61344/run_gbEuxN/runtest
Import file transfer initiated. Wed Jul  7 11:26:05 2021
Import file transfer complete. Wed Jul  7 18:26:05 2021


#### Return code
The return code (here called `res`) for normal completion is zero, which we can check on here:

In [32]:
res

0

#### Standard out
Standard out from our call is as follows:

In [33]:
stdout

b'=SUBMIT-PROGRESS=> aborted=0 finished=0 failed=0 executing=0 waiting=2 setting_up=0 setup=0 %done=0.00 timestamp=1625682283.8\n=SUBMIT-PROGRESS=> aborted=0 finished=0 failed=0 executing=2 waiting=0 setting_up=0 setup=0 %done=0.00 timestamp=1625682319.0\n=SUBMIT-PROGRESS=> aborted=0 finished=2 failed=0 executing=0 waiting=0 setting_up=0 setup=0 %done=100.00 timestamp=1625682354.2\nSimulations complete. Results are stored in directory /home/vhub/jsperhac/data/sessions/61344/run_gbEuxN/runtest\nImport file transfer initiated. Wed Jul  7 11:26:05 2021\nImport file transfer complete. Wed Jul  7 18:26:05 2021\n'

#### Standard error
Finally, here is the standard error, `stderr`:

In [34]:
stderr

b''

#### Results of the calculation

For the last example, we used a test program installed on our hub and asked submit to execute it on the remote resource.  The program, `sim1.py`, takes a single argument and outputs values to a file, `out.log`.  It does not write the results to stdout.  You can see all the output in the subdirectories under `runtest`, as follows:

In [35]:
!ls -lR runtest

runtest:
total 4
drwxr-xr-x 2 jsperhac public  46 Jul  7 18:26 01
drwxr-xr-x 2 jsperhac public  46 Jul  7 18:26 02
-rw-r--r-- 1 jsperhac public 225 Jul  7 18:25 parameterCombinations.csv

runtest/01:
total 4
-rw-r--r-- 1 jsperhac public 1660 Jul  7 18:25 out.log
-rw-r--r-- 1 jsperhac public    0 Jul  7 18:25 runtest_01.stdout

runtest/02:
total 4
-rw-r--r-- 1 jsperhac public 1566 Jul  7 18:25 out.log
-rw-r--r-- 1 jsperhac public    0 Jul  7 18:25 runtest_02.stdout


If the run was unsuccessful, the command above will list the .stderr files that will indicate the source of the error.
Alternately, if the run was successful, we can look at the output generated by the simulation and saved in the file `out.log`:

In [36]:
!cat runtest/01/out.log

0 0
0.606061 0.0588061
1.21212 0.114154
1.81818 0.166247
2.42424 0.215277
3.0303 0.261423
3.63636 0.304856
4.24242 0.345735
4.84848 0.38421
5.45455 0.420422
6.06061 0.454504
6.66667 0.486583
7.27273 0.516775
7.87879 0.545191
8.48485 0.571937
9.09091 0.59711
9.69697 0.620802
10.303 0.643101
10.9091 0.664089
11.5152 0.683843
12.1212 0.702435
12.7273 0.719933
13.3333 0.736403
13.9394 0.751904
14.5455 0.766494
15.1515 0.780225
15.7576 0.793149
16.3636 0.805313
16.9697 0.816762
17.5758 0.827538
18.1818 0.837679
18.7879 0.847225
19.3939 0.856209
20 0.864665
20.6061 0.872623
21.2121 0.880114
21.8182 0.887164
22.4242 0.893799
23.0303 0.900045
23.6364 0.905922
24.2424 0.911455
24.8485 0.916662
25.4545 0.921563
26.0606 0.926175
26.6667 0.930517
27.2727 0.934603
27.8788 0.938448
28.4848 0.942068
29.0909 0.945475
29.697 0.948681
30.303 0.951699
30.9091 0.954539
31.5152 0.957213
32.1212 0.959729
32.7273 0.962097
33.3333 0.964326
33.9394 0.966424
34.5455 0.968398
35.1515 0.970257
35.7576 0.972006
36

## Submit status email
A special application called `mail2self` enables you to send email from submit to the current user. This feature is handy to use for alerting the user once their job has returned its results from the remote computing resource.

This simple example below shows how it is done. Here we set up the call:

In [27]:
email_text = "Test email from the jupyterexamples tool, using mail2self and submit"
email_subject = "Submit mail2self test"
email_cmd = 'submit --progress silent mail2self -t "'+email_text+'" -s "'+email_subject+'"'

Execute the `os.system()` call, then check the email address you used for your Hub registration. A return value of 0 indicates success:

In [26]:
os.system(email_cmd)

0

# Using SubmitCommand class

Furthur Python integration is achieved by using the hubzero.submit.SubmitCommand class.  Methods are provided for specifying any and all **submit** arguments.  For each **submit** command argument there are typically two method types - set and reset. The standard python **help** builtin can be used to show methods and associated arguments.  The code shown here can also be used Rappture Python wrapper scripts.

<div class="alert alert-warning">
<h3>Note</h3>
The <i>hubzero.submit</i> package is not currently available from Jupyter on this Hub. The documentation that follows is provided for reference only.
</div>

In [ ]:
from hubzero.submit.SubmitCommand import SubmitCommand

submitCommand = SubmitCommand()
help (submitCommand)

Help about the underlying **submit** command can be easily reported.

In [ ]:
submitCommand = SubmitCommand()
result = submitCommand.submit(['--help'])

A listing of available venues can be listed

In [ ]:
submitCommand = SubmitCommand()
submitCommand.setHelp(detail='venues')
result = submitCommand.submit()

A listing of available staged tools can be listed

In [ ]:
submitCommand = SubmitCommand()
submitCommand.setHelp(detail='tools')
result = submitCommand.submit()

For demonstration purposes we can use a Python application that is part of the Pegasus Tutorial tool. Because it is installed in the /apps directory it is available for submisison to most venues.

In [ ]:
applicationCode = '/apps/pegtut/current/examples/capacitor_voltage/sim1.py'

The SubmitCommand class has a method that excepts command arguments as a simple list.

In [ ]:
submitCommand = SubmitCommand()
result = submitCommand.submit(['-w','5',applicationCode,'--C','0.001','--Vin','3'])
print(result)

You should notice that the single run resulted in submission of jobs to three different sites.  When no venue is specified and the application is not staged at any remote venues redundant jobs are submited.  When one job completes sucessfully the other jobs are terminated.
The result dictionary has three members.  The *jobId* is the unique identifier given by **submit** to every run.  *runName* is the name of the run and by default it matches the *jobId*.  *runName* can be optionally set with **submit** arguments.  Standard output and standard error files generated by remote application execution are returned as *runName*.stdout and *runName*.stderr.  The standard error file will not be returned if it is empty. 

In [ ]:
runStdout = result['runName'] + '.stdout'
!cat $runStdout

In [ ]:
runStderr = result['runName'] + '.stderr'
!cat $runStderr

Using the set methods and specifying a particular venue

In [ ]:
submitCommand = SubmitCommand()
submitCommand.setWallTime(5)
submitCommand.setVenue('OSGFactory')
submitCommand.setCommand(applicationCode)
submitCommand.setCommandArguments(['--C','0.001','--Vin','3'])
submitCommand.show()
result = submitCommand.submit()

Here is an example with a parameter sweep over two variables.  Parameter sweep results are returned in a directory named *runName*.  Each job in the sweep has a separate subdirectory.

In [ ]:
submitCommand = SubmitCommand()
submitCommand.setWallTime(5)
submitCommand.setParameters(['@@Vin=1:2:10','@@C=100e-6,100e-5'])
submitCommand.setCommand(applicationCode)
submitCommand.setCommandArguments(['--C','@@C','--Vin','@@Vin'])
submitCommand.show()
result = submitCommand.submit()

runResultsDir = result['runName']
!ls -R $runResultsDir

Alternative progress reporting methods are provided to facilitate advanced reporting. As an example the *submit* progress reporting method output is injested by Rappture and rendered as a progress bar.

In [ ]:
submitCommand = SubmitCommand()
submitCommand.setWallTime(5)
submitCommand.setParameters(['@@Vin=1:2:10','@@C=100e-6,100e-5'])
submitCommand.setProgress(detail='submit')
submitCommand.setCommand(applicationCode)
submitCommand.setCommandArguments(['--C','@@C','--Vin','@@Vin'])
submitCommand.show()
result = submitCommand.submit()

Parameter sweep variable values can also be read from a file.

In [ ]:
with open("input.csv",'w') as fpInput:
   fpInput.write("@@Vin,@@C\n")
   for v in range(1,6):
      for c in (0.0001,0.001):
         fpInput.write("%f,%f\n" % (v,c))
!cat input.csv

In [ ]:
submitCommand = SubmitCommand()
submitCommand.setWallTime(5)
submitCommand.setDataFile('input.csv')
submitCommand.setCommand(applicationCode)
submitCommand.setCommandArguments(['--C','@@C','--Vin','@@Vin'])
submitCommand.show()
result = submitCommand.submit()

The mapping of variable combinations to job number is contained in the parameterCombinations.csv file.

In [ ]:
import os.path
combinationsFile = os.path.join(result['runName'],'parameterCombinations.csv')
!cat $combinationsFile